Draw boundary polygon on map to discover all DBpedia resources within the boundary shape using a federated search. Uses the ipyleaflet python package.

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)
from traitlets import link

def wkt_convert(c):
    return str(c[0]) + " " + str(c[1])

def handle_draw(self, action, geo_json):
    global m
    coords = geo_json['geometry']['coordinates'][0]
    wkt = "POLYGON((%s " % ', '.join(map(wkt_convert, coords)) + "))"
    sparql_query = """
                    SELECT DISTINCT ?resource ?label ?long ?lat
                    WHERE 
                    {{ 
                        SERVICE <http://dbpedia.org/sparql>
                        {{
                                ?resource geo:geometry ?location ;
                                          rdfs:label ?label . 
                                FILTER( bif:st_intersects( ?location, bif:st_geomfromtext({}),0.0)) .
                                FILTER (lang(?label) = 'en') .
                                BIND(bif:st_x(?location) AS ?long) .
                                BIND(bif:st_y(?location) AS ?lat) .
                        }}
                    }}
                    """.format("\"{}\"".format(wkt))
    
    # Assumes OpenLink Virtuoso sparql endpoint running locally at:
    sparql_endpoint = "http://localhost:8890/sparql"

    sparql = SPARQLWrapper(sparql_endpoint)
    sparql.setQuery(sparql_query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    for result in results["results"]["bindings"]:
        mark = Marker(location=[result['lat']['value'], result['long']['value']], title=result['label']['value'])
        m.add_layer(mark)

center = [54.047186,-2.801706]
zoom = 15
m = Map(default_tiles=TileLayer(opacity=1.0), center=center, zoom=zoom)
dc = DrawControl()
dc.on_draw(handle_draw)
m.add_control(dc)
m